<a href="https://colab.research.google.com/github/minus--/tensorflow-tutorials/blob/master/simple_seq2seq_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sequence To Sequence Model in Tensorflow 2.0
### French to English translation

### Imports

In [24]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import random

### Data Download and Processing

In [4]:
# Download the file
!([ -f fra-eng.zip ] && rm *.zip && rm *.txt  || echo "No existing file found") && wget http://www.manythings.org/anki/fra-eng.zip && unzip fra-eng.zip

path_to_zip = os.path.dirname(os.path.realpath('fra-eng.zip'))+"/fra-eng.zip"
path_to_file = os.path.dirname(os.path.realpath('fra.txt'))+"/fra.txt"


--2019-04-11 08:04:17--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.109.196, 104.24.108.196, 2606:4700:30::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.109.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3423204 (3.3M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   3.26M  13.0MB/s    in 0.3s    

2019-04-11 08:04:18 (13.0 MB/s) - ‘fra-eng.zip’ saved [3423204/3423204]

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [0]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [6]:
en_sentence = u"May I borrow this book?"
fr_sentence = u"je peux vous empreinter un livre?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(fr_sentence).encode('utf-8'))

<start> may i borrow this book ? <end>
<start> je peux vous empreinter un livre ? <end>


In [0]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, FRENCH]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    
    return zip(*word_pairs)

In [8]:
en, fr = create_dataset(path_to_file, None)
print(en[-1])
print(fr[-1])

<start> it may be impossible to get a completely error free corpus due to the nature of this kind of collaborative effort . however , if we encourage members to contribute sentences in their own languages rather than experiment in languages they are learning , we might be able to minimize errors . <end>
<start> il est peut etre impossible d obtenir un corpus completement denue de fautes , etant donnee la nature de ce type d entreprise collaborative . cependant , si nous encourageons les membres a produire des phrases dans leurs propres langues plutot que d experimenter dans les langues qu ils apprennent , nous pourrions etre en mesure de reduire les erreurs . <end>


In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)
  
  tensor = lang_tokenizer.texts_to_sequences(lang)
  
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
  
  return tensor, lang_tokenizer

In [0]:
def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
    targ_lang, inp_lang = create_dataset(path, num_examples)

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [0]:
# Try experimenting with the size of that dataset
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [13]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(24000, 24000, 6000, 6000)

In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [15]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
22 ----> tom
50 ----> aime
17 ----> le
4016 ----> fromage
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
16 ----> tom
173 ----> likes
2754 ----> cheese
3 ----> .
2 ----> <end>


In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

### Encoder Definition

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    
    self.lstm = tf.keras.layers.LSTM(self.enc_units,
                                     return_sequences=True, 
                                     return_state=True, 
                                     recurrent_initializer='glorot_uniform')


  def call(self, x, hidden):
    x = self.embedding(x)
    output, state_h,state_c = self.lstm(x, initial_state=hidden)        
    return output, state_h, state_c 

  def initialize_hidden_state(self):
    return [tf.zeros((self.batch_sz, self.enc_units)),tf.zeros((self.batch_sz, self.enc_units))]
    #return self.lstm.zero_state()

In [18]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 17]), TensorShape([64, 10]))

In [19]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden_h, sample_hidden_c = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden_h.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden_c.shape))

W0411 08:04:48.385406 140457399551872 recurrent.py:2914] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fbe4c044350>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Encoder output shape: (batch size, sequence length, units) (64, 17, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


### Decoder Definition

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(self.dec_units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

  def call(self, x, hidden, enc_output):     

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # passing the concatenated vector to the GRU
    output, h_state, c_state = self.lstm(x, initial_state=hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, h_state , c_state

In [27]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)), 
                                      [sample_hidden_h, sample_hidden_c], sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

W0411 08:19:24.231101 140457399551872 recurrent.py:2914] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fbe4bb1bad0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


Decoder output shape: (batch_size, vocab size) (64, 4442)


### Loss Function

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

### Training Loop

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden, teacher_forcing_ratio = 0.5):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden_h, enc_hidden_c = encoder(inp, enc_hidden)

    dec_hidden = [enc_hidden_h, enc_hidden_c]

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)       

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      teacher_force = random.random() < teacher_forcing_ratio

      # passing enc_output to the decoder
      predictions, dec_hidden_h, dec_hidden_c = decoder(dec_input, dec_hidden, enc_output)
      dec_hidden = [dec_hidden_h, dec_hidden_c]

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      predict_ids = tf.expand_dims(tf.argmax(predictions, axis=1), 1)

      dec_input = tf.expand_dims(targ[:, t], 1) if teacher_force else predict_ids

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))
  
  return batch_loss

### Run the Training

In [66]:
EPOCHS = 30

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.8929
Epoch 1 Batch 100 Loss 2.1008
Epoch 1 Batch 200 Loss 1.8143
Epoch 1 Batch 300 Loss 1.8731
Epoch 1 Loss 1.8189
Time taken for 1 epoch 46.9882180691 sec

Epoch 2 Batch 0 Loss 1.7891
Epoch 2 Batch 100 Loss 1.5851
Epoch 2 Batch 200 Loss 1.5215
Epoch 2 Batch 300 Loss 1.5720
Epoch 2 Loss 1.5637
Time taken for 1 epoch 47.5436458588 sec

Epoch 3 Batch 0 Loss 1.4696
Epoch 3 Batch 100 Loss 1.3576
Epoch 3 Batch 200 Loss 1.3232
Epoch 3 Batch 300 Loss 1.3813
Epoch 3 Loss 1.3596
Time taken for 1 epoch 46.9673991203 sec

Epoch 4 Batch 0 Loss 1.3267
Epoch 4 Batch 100 Loss 1.1842
Epoch 4 Batch 200 Loss 1.1846
Epoch 4 Batch 300 Loss 1.2275
Epoch 4 Loss 1.2164
Time taken for 1 epoch 47.746309042 sec

Epoch 5 Batch 0 Loss 1.2127
Epoch 5 Batch 100 Loss 1.0578
Epoch 5 Batch 200 Loss 1.0743
Epoch 5 Batch 300 Loss 1.1228
Epoch 5 Loss 1.0968
Time taken for 1 epoch 47.0672519207 sec

Epoch 6 Batch 0 Loss 1.1038
Epoch 6 Batch 100 Loss 0.9467
Epoch 6 Batch 200 Loss 0.9559
Epoch 6 Batch

### Evaluate the Results

In [0]:
def evaluate(sentence):    
    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], 
                                                           maxlen=max_length_inp, 
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units)), tf.zeros((1, units))]
    enc_out, enc_hidden_h, enc_hidden_c  = encoder(inputs, hidden)

    dec_hidden = [enc_hidden_h, enc_hidden_c]
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
    
    for t in range(max_length_targ):
        predictions, dec_hidden_h, dec_hidden_c = decoder(dec_input, 
                                                             dec_hidden, 
                                                             enc_out)
        dec_hidden = [dec_hidden_h, dec_hidden_c]
        
        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [0]:
def translate(sentence):
    result, sentence = evaluate(sentence)
        
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    

In [69]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [70]:
translate(u"c'est ma vie")

Input: <start> c est ma vie <end>
Predicted translation: that s my sister . <end> 


In [71]:
translate(u"je suis a la maison")

Input: <start> je suis a la maison <end>
Predicted translation: i m at home . <end> 


In [72]:
translate(u"je suis dans les environs")

Input: <start> je suis dans les environs <end>
Predicted translation: i m on strike . <end> 


In [73]:
translate(u"tu es encore a la maison?")

Input: <start> tu es encore a la maison ? <end>
Predicted translation: whose son is tom ? <end> 


In [74]:
translate(u"il fait tres froid")

Input: <start> il fait tres froid <end>
Predicted translation: it s very cold . <end> 


In [75]:
translate("je peux vous acheter un livre ?")

Input: <start> je peux vous acheter un livre ? <end>
Predicted translation: can i help you you ? <end> 
